<a href="https://colab.research.google.com/github/isegura/OCW-UC3M-NLPDeep-2023/blob/main/tema4_1_spacyWE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png" width=50%/>

<h1><font color='#12007a'>Procesamiento de Lenguaje Natural con Aprendizaje Profundo</font></h1>
<p>Autora: Isabel Segura Bedmar</p>

<img align='right' src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>    

# 4.1. Cómo utilizar los word embeddings de Spacy.

Spacy proporciona diferentes modelos (https://spacy.io/usage/models) que permiten realizar tareas  de PLN como la tokenización, la lematización, el análisis de dependencias, etc.

Algunos de estos modelos además incluyen vectores de palabras que pueden ser utilizados para medir la similitud entre palabras o entre textos.

Los modelos que terminan en -sm (small) no incluyen vectores de palabras.


El primer paso será instalar la librería spacy:




In [1]:
!pip install -q spacy

A continuación, vamos a descargar y cargar uno de estos modelos. Por ejemplo, el modelo **es_core_news_md** para procesar textos en español (optimizado para CPU). Este modelo incluye los siguientes componentes: tok2vec, morphologizer, parser, senter, ner, attribute_ruler, lemmatizer. Además, incluye un modelo de vectores con unos 20.000 vectores. Los vectores tienen dimensión 300.

In [2]:
!python3 -m spacy download es_core_news_md

2023-09-27 07:56:13.663267: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 07:56:16.610048: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 17.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [3]:
import spacy
nlp = spacy.load("es_core_news_md")


Ahora vamos a utilizar dicho modelo para procesar algunas palabras y estudiar sus embeddings. Es decir, vamos a aplicar el objeto nlp, creado en la celda anterior, para procesar una secuencia de palabras. De cada palabra, vamos a consultar los siguientes atributos:
- **text**: el texto del token (palabra).
- **has_vector**: atributo booleano que nos indica si el token tiene un vector o no.
- vector_norm: nos devuelve la norma del vector (la raíz cuadrada de la suma de los cuadrados de sus elementos).
- is_oov: es True si la palabra forma parte del vocabulario y False en otro caso.

In [4]:
tokens = nlp("caballo yegua potro vaca manzana naranja iansdiufnas Falcado")

for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

caballo True 22.985107 False
yegua True 26.477182 False
potro True 22.985107 False
vaca True 30.985302 False
manzana True 19.44172 False
naranja True 18.938 False
iansdiufnas False 0.0 True
Falcado False 0.0 True


Las únicas palabras que no tienes vector asociado son: **iansdiufnas**, que claramente no existe, y **falcado**, una palabra poco común en español y cuyo significado es curvado con forma de hoz.


Para poder acceder al embedding, directamente debemos invocar el atributo **vector**:

In [5]:
token1=tokens[0]
print("dimensión: ", len(token1.vector))
print(token1.text, ":", token1.vector)

dimensión:  300
caballo : [ 1.6796e+00  6.9239e-01 -1.0369e+00  2.2072e+00  1.1877e+00 -1.2560e+00
  1.6072e-01  1.2165e+00 -8.3058e-02  2.9497e-01 -1.8744e+00  8.9574e-01
 -2.7154e-01 -5.5810e-01 -1.1556e+00 -2.8373e+00 -2.6771e+00 -7.0166e-02
  5.4694e-02  9.0240e-01 -1.5919e-01 -7.6455e-02  3.2906e-01  8.3176e-01
  6.7742e-01 -1.6503e+00  6.5548e-01 -5.0972e-01  7.3731e-01  1.9637e+00
 -1.8252e+00  1.0735e-01  1.5032e+00  5.8705e-01 -2.8360e+00 -1.0602e+00
  8.6917e-01 -4.9286e-01  1.6447e+00 -1.2131e+00 -2.0851e-01  1.1079e+00
 -2.8351e-01  3.9133e+00 -2.1549e+00 -9.1772e-01  1.3531e+00 -1.9251e+00
 -8.3949e-01 -1.4306e+00  4.5392e-01  9.9385e-01  9.6361e-01 -4.0914e-01
 -2.4818e+00 -1.2071e+00 -5.1422e-01 -8.5908e-01  1.8626e+00  2.9035e+00
  1.8261e-01  1.7438e+00  2.3146e+00 -7.7685e-01  1.9547e+00 -7.9520e-02
  4.3168e-01 -6.7273e-01 -2.4355e+00 -4.1766e-02 -4.3220e-01 -7.5098e-01
  2.3693e-01  1.2240e+00  1.9430e+00  1.2342e+00  7.1813e-01  2.5070e+00
 -2.2650e+00 -2.3766e+00 

Spacy nos proporciona un método para calcular la **similitud** (similarity) entre dos tokens (también puede ser utilizado para calcular la similitud entre dos textos). El método está basado en la **distancia del coseno**.

El método devuelve un valor entre 0 y 1, donde un valor próximo a 1 significa que ambas tokens (o textos) tienen un significado muy similar similar, mientras un valor próximo a 0 indica que ambos tokens (o textos) no guardan similitud.

En la siguiente celda, vamos a calcular la similitud de la primera palabra (**caballo**) con el resto de palabras. ¿Cuál es la palabra con mayor similitud?.

In [6]:
token1=tokens[0]
for token in tokens[1:]:
    print(token1,token,token1.similarity(token))


caballo yegua 0.3896075189113617
caballo potro 0.9999998807907104
caballo vaca 0.4315049350261688
caballo manzana 0.19715562462806702
caballo naranja 0.19350798428058624
caballo iansdiufnas 0.0
caballo Falcado 0.0


<ipython-input-6-13b710075a3d>:3: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  print(token1,token,token1.similarity(token))


También podemos comparar dos textos.

In [7]:
doc1 = nlp("¿Cómo puedo desinstalar la aplicación?")
doc2 = nlp("¿Cómo puedo encontrar el verdadero amor?")
doc1.similarity(doc2)

0.7478227116063041

En el anterior ejemplo, la similitud tiene un valor de 0.74, sin embargo, el significado de las palabras es completamente diferente.  
Probablemente esa alta similitud se ha obtenido porque ambas oracioens comparten el mismo inicio (**¿Cómo puedo**). Si eliminamos ese comienzo, la similitud baja.


In [8]:
doc1 = nlp("desinstalar la aplicación?")
doc2 = nlp("encontrar el verdadero amor?")
doc1.similarity(doc2)

0.31684383552078227

In [9]:
doc1 = nlp("desinstalar la aplicación?")
doc2 = nlp("eliminar el software?")
doc1.similarity(doc2)

0.4280196172067276

Spacy incluye diferentes modelos con vectores de palabras. Estos vectores de palabras nos proporcionan un método sencillo para representar los textos, sino que además pueden ser utilizados directamente para calcular la similitud entre ellos.